# Spatial Operations in GeoPandas

In this section of the tutorial, we delve into how to use GeoPandas to perform various spatial operations on geometries. These operations include:
- calculating areas
- checking for spatial relationships like intersection or containment
- creating new geometries such as centroids, buffers, convex hulls, and more.

The goal is to understand how to explore and analyze geographic data using the powerful tools provided by GeoPandas.

We will use real-world datasets from New York City's open data portal. These include:

* Locations of schools (as point data).
* Subway entrances (also as points).
* Bike paths (as line geometries).
* Parks (as polygon geometries).

These datasets will help us explore a variety of geometry types and spatial relationships.

Once loaded, we visualize these datasets using the `.explore()` method, which provides an interactive map to verify and inspect spatial data. This helps ensure that our data looks as expected before performing any operations.

A key preliminary step is setting the correct Coordinate Reference System (CRS). We use `.to_crs()` for this purpose. 

Many geospatial calculations such as distance or area depend on the CRS being appropriate for the unit of measurement. For example, the common EPSG:4326 uses degrees, but if we want accurate distance or area calculations in meters or feet, we switch to EPSG:3857 or EPSG:2263. 

The tutorial demonstrates how to calculate the area of each neighborhood by accessing the `.area` property of the geometry column. We store this as a new column in the GeoDataFrame.

We then extract the boundary of each park using the `.boundary` property. This is useful when you only want to work with the outer edge of a polygon.

Next, we calculate the lengths of bike paths, showcasing how projection affects measurements (e.g., meters vs feet). By changing the projection with `.to_crs()`, we see how GeoPandas returns different values because it uses the underlying CRS units.

We also explore filtering specific features using attribute queries. For instance, we filter to a specific bike segment ID and visualize it. The tutorial emphasizes using the correct CRS when mapping or measuring to get consistent and interpretable results.

Further analysis involves calculating distances. One example is calculating the shortest distance between schools and subway entrances. This is done using the `.distance()` method along with `.apply()` and a `lambda` function to find the minimum distance for each school.

We also explore geometric operations like:

* `.centroid`: Finding the centroid of a polygon.
* `.buffer()`: Creating buffers around features (e.g., 10 meters around bike paths).
* `.intersects()`: Checking whether geometries intersect.
* `.contains()` and `.within()`: Checking containment relationships.
* `.bounds`, `.convex_hull`, and `.envelope`: Calculating bounding rectangles, convex hulls, and other spatial summaries.
* `.is_valid`: Checking geometry validity.
* `.offset_curve`: Translating geometries (e.g., shifting subway entrance points).

We apply these methods to answer spatial questions, such as:

* Which parks intersect with bike paths?
* Which neighborhoods contain parks entirely?
* Which schools are within 500 meters of a subway entrance?
* Which parks overlap with multiple neighborhoods?
* Which features fall outside any neighborhood?

Finally, the section highlights how GeoPandas supports powerful, low-code spatial analytics. Although this is not an exhaustive list of spatial functions, it provides a broad toolkit for working with geospatial data. For a full reference, the instructor recommends reviewing the GeoPandas documentation.

Now that we've covered core geometry operations, we are well-equipped to move forward with more advanced spatial joins and analytical workflows that will appear in subsequent parts of the tutorial.


In [ ]:
import geopandas as gpd

In [ ]:
path = '../../geopandas_101_DATA/Forrest/data/nyc'

schools = gpd.read_file(f"{path}/SchoolPoints_APS_2024_08_28 (1)/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file(f"{path}/nyc_subway_entrances/nyc_subway_entrances.shp")
bike_paths = gpd.read_file(f"{path}/New York City Bike Routes_20241223.geojson")
parks = gpd.read_file(f"{path}/Parks Properties_20241223.geojson")
hoods = gpd.read_file(f"{path}/nyc_neighborhoods.geojson")

In [ ]:
schools.explore()

In [ ]:
subways.explore()

In [ ]:
bike_paths.explore()

In [ ]:
parks.explore()

In [ ]:
hoods.explore()

We need to have the same CRS in all dataframes. We will set it to 3857. This CRM measures in meters.

In [ ]:
frames = [schools, subways, bike_paths, hoods, parks]
for frame in frames:
    print(frame.crs)

In [ ]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
hoods = hoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

In [ ]:
frames = [schools, subways, bike_paths, hoods, parks]
for frame in frames:
    print(frame.crs)

In [ ]:
for frame in frames:
    print(frame.geom_type.unique())

In [ ]:
import pandas as pd

pd.options.display.float_format = '{:,.4f}'.format

## Area

Extract the area of each neighborhood from the ```geometry``` column. It is measureed in square meters.

In [ ]:
# 1. GeoSeries.area
hoods["area"] = hoods.geometry.area

head = hoods.sort_values(['area'], ascending=False).head()
display(head)
head.explore()

## Boundary

In [ ]:
hoods.boundary.explore()

In [ ]:
# 2. GeoSeries.boundary
hoods['boundary'] = hoods['geometry'].boundary

In [ ]:
hoods['boundary'].explore()

In [ ]:
# 2. GeoSeries.boundary
parks["boundary"] = parks.geometry.boundary

# Create a single geoseries with a different CRS
parks_boundary = parks['boundary'].to_crs('EPSG:4326')

# parks.explore()
# parks.geometry.explore()

parks.boundary.explore()

In [ ]:
type(parks)

In [ ]:
type(parks_boundary)

Notice that parks now has two geometry type variables: 'geometry' and 'boundary'

GeoPandas can only have one active geometry column at a time — the one that defines the spatial behavior of the DataFrame (e.g., used for plotting, spatial joins, CRS operations).

So while gdf now contains two columns with geometry-like data, only one is recognized by GeoPandas as the "active" geometry.

The active geometry column is usually named "geometry".

Your new "boundary" column simply stores geometry objects (usually LineString types), but is treated like a regular data column unless you explicitly set it as the active geometry.

To know which is the active geometry column: ```gdf.geometry.name```

If you want GeoPandas to use "boundary" as the main geometry: ```gdf.set_geometry('boundary')```
)

Now gdf.geometry.name will return 'boundary', and all spatial methods like .plot(), .to_crs(), or .area will apply to that column.

In [ ]:
parks.geometry.name

In [ ]:
parks.set_geometry('boundary').geometry.name

Also notice that the boundary column doesn't contain polygons anymore, but lines (the polygon's boundary lines)

In [ ]:
print(parks.geom_type.unique())
print(parks['boundary'].geom_type.unique())

## Bounds

The ```.bounds``` property in GeoPandas returns the bounding box (or extent) of each geometry in your GeoDataFrame or GeoSeries.
A bounding box is the smallest rectangle (aligned with the coordinate axes) that completely encloses a geometry.

```.bounds``` returns a regular Pandas (not GeoPandas) data frame.

In [ ]:
parks.crs

In [ ]:
parks_4326 = parks.to_crs(epsg=4326)

In [ ]:
# parks.geometry.bounds
# parks.boundary.bounds
parks.bounds

In [ ]:
parks_4326.bounds

In [ ]:
parks.total_bounds

In [ ]:
parks_4326.total_bounds

In [ ]:
from shapely.geometry import box

# Create bounding boxes as geometries
bounds = parks_4326.bounds
boxes = [box(xmin, ymin, xmax, ymax) for xmin, ymin, xmax, ymax in bounds.values]

# Turn bounding boxes into a GeoDataFrame
bounds_gdf = gpd.GeoDataFrame(geometry=boxes, crs=parks_4326.crs)

# Plot original geometries with bounds overlaid
ax = parks_4326.plot(edgecolor='black', facecolor='lightgray', linewidth=0.5)
bounds_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=0.5)

In [ ]:
from shapely.geometry import box

# Create bounding boxes as geometries
bounds = parks.bounds
boxes = [box(xmin, ymin, xmax, ymax) for xmin, ymin, xmax, ymax in bounds.values]

# Turn bounding boxes into a GeoDataFrame
bounds_gdf = gpd.GeoDataFrame(geometry=boxes, crs=parks.crs)

# Plot original geometries with bounds overlaid
ax = parks.plot(edgecolor='black', facecolor='lightgray', linewidth=0.5)
bounds_gdf.plot(ax=ax, edgecolor='red', facecolor='none', linewidth=0.5)

In [ ]:
bounds_gdf.explore()

In [ ]:
parks.crs

In [ ]:
parks_4326.crs

In [ ]:
parks.length
parks.boundary.length

## Length

[EPSG 2263](https://epsg.io/?q=2263) is specific to New York state and Long Island. It measures distances in feet.

In [ ]:
paths_2263 = bike_paths.to_crs('EPSG:2263')

In [ ]:
# The geometry of this frame contains only lines, so all areas should be zero
assert (paths_2263.area==0).all()

In [ ]:
paths_2263["length"] = paths_2263.geometry.length

In [ ]:
# Plot the longest bike path in NYC
mask = paths_2263['length']==paths_2263['length'].max()
paths_2263.loc[mask].buffer(100).explore()

In [ ]:
schools["nearest_subway_distance"] = schools.geometry.apply(
    lambda school: subways.distance(school).min()
)
# or:
# schools["nearest_subway_distance"] = [subways.distance(g).min() for g in schools.geometry]

In [ ]:
schools[['Name', 'nearest_subway_distance']].sort_values(by=['nearest_subway_distance'], ascending=True)

In [ ]:
schools[schools['Name'].isin(['P.S. 150', 'P.S. 094 David D. Porter'])].buffer(500).explore()

In [ ]:
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Plot all schools as blue points
schools.plot(ax=ax, color='blue', markersize=5, label='Schools')

# Plot all subway stations as red points
subways.plot(ax=ax, color='red', markersize=8, label='Subway Stations')

# Add a legend and title
plt.legend()
plt.title("Schools and Subway Stations in the City", fontsize=14)
plt.xlabel("Longitude")
plt.ylabel("Latitude")

plt.show()

In [ ]:
hoods.explore()

In [ ]:
hoods.centroid.explore()

In [ ]:
bike_paths.geometry.buffer(10)

In [ ]:
bike_paths.geometry

In [ ]:
bike_paths.head(100).geometry.explore()

In [ ]:
bike_paths.head(100).geometry.buffer(100).explore()

In [ ]:
parks["intersects_bike_path"] = parks.geometry.apply(
    lambda park: bike_paths.geometry.intersects(park).any()
)

In [ ]:
parks[parks['intersects_bike_path'] == True].explore()

In [ ]:
parks.explore()

In [ ]:
parks.minimum_bounding_circle().explore()

In [ ]:
parks[parks['eapply']=="Central Park"].minimum_bounding_circle().explore()

In [ ]:
parks[parks['eapply']=="Central Park"].envelope.explore()

In [ ]:
parks[parks['eapply']=="Central Park"].boundary.explore()

In [ ]:
hoods[hoods['neighborhood']=="Upper West Side"].geometry.explore()

In [ ]:
hoods[hoods['neighborhood']=="Upper West Side"].geometry.simplify(tolerance=0.001).explore()

In [ ]:
hoods[hoods['neighborhood']=="Upper West Side"].geometry.convex_hull.explore()

In [ ]:
hoods[hoods['neighborhood']=="Upper West Side"].geometry.envelope.explore()

In [ ]:
hoods[hoods['neighborhood']=="Upper West Side"].geometry.minimum_bounding_circle().explore()

In [ ]:
hoods.geometry.convex_hull.explore()

In [ ]:
schools.geometry.envelope.explore()

In [ ]:
schools.geometry.envelope.explore()

In [ ]:
parks_neighborhoods_overlay = hoods.overlay(parks, how="intersection")

In [ ]:
parks_neighborhoods_overlay

In [ ]:
print(parks['geometry'])
print(hoods['geometry'])
print(parks_neighborhoods_overlay['geometry'])

In [ ]:
parks.boundary.explore()

In [ ]:
parks.geometry.boundary.explore()

In [ ]:
print(len(parks_neighborhoods_overlay))
print(len(hoods))

In [ ]:
parks_neighborhoods_overlay.explore()

In [ ]:
neighborhoods_parks_overlay.explore()

In [ ]:
neighborhoods_parks_overlay = parks.overlay(hoods, how="intersection")

In [ ]:
neighborhoods_parks_overlay

In [ ]:
subways["translated"] = subways.geometry.translate(xoff=1000, yoff=1000)
subways.translated.explore()

In [ ]:
parks.geometry.is_valid

In [ ]:
hoods.geometry.exterior.explore()

In [ ]:
bike_paths.geometry.geom_type


In [ ]:
schools["x"] = schools.geometry.to_crs(epsg=4326).x
schools["y"] = schools.geometry.to_crs(epsg=4326).y

In [ ]:
schools.info()

In [ ]:
schools[['x', 'Longitude', 'y', 'Latitude']]

In [ ]:
hoods["contains_parks"] = hoods.geometry.apply(
    lambda neighborhood: parks.geometry.apply(
        lambda park: neighborhood.contains(park)
    ).any()
)

In [ ]:
hoods[hoods['contains_parks'] == False].explore()

In [ ]:
bike_paths["crosses_parks"] = bike_paths.geometry.apply(
    lambda path: path.crosses(parks.geometry).any()
)

In [ ]:
bike_paths[bike_paths['crosses_parks'] == True].explore()

In [ ]:
parks["disjoint_neighborhoods"] = parks.geometry.apply(
    lambda park: park.disjoint(hoods.geometry).all()
)

In [ ]:
parks[parks['disjoint_neighborhoods'] == True].explore()

In [ ]:
def schoolsWithin500(school):
    return school.distance(subways.geometry).any() <= 500

In [ ]:
schools["within_500m_subway"] = schools.geometry.apply(
    schoolsWithin500
)


In [ ]:
subway_union = subways.geometry.union_all()
schools["within_500m_subway"] = schools.geometry.distance(subway_union) <= 500


In [ ]:
schools[schools["within_500m_subway"] == True].explore()

In [ ]:
parks["overlaps_neighborhoods"] = parks.geometry.apply(
    lambda park: park.overlaps(hoods.geometry).any()
)


In [ ]:
parks[parks["overlaps_neighborhoods"] == True].explore()

In [ ]:
parks['geometry'].symmetric_difference(hoods['geometry']).explore()